In [1]:
from model import *
from mm import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
import ast

def convert_to_numeric(str_list):
    # Use ast.literal_eval to safely convert string to actual list with numeric entries
    numeric_list = ast.literal_eval(str_list)
    return numeric_list




In [3]:
data_raw = pd.read_csv('historical_ETHUSDT_orderbook_data_2.csv')
data_raw["Timestamp"] = pd.to_datetime(data_raw["Timestamp"])
data_raw['Bid List'] = data_raw['Bid List'].apply(convert_to_numeric)
data_raw['Bid Size List'] = data_raw['Bid Size List'].apply(convert_to_numeric)
data_raw['Ask List'] = data_raw['Ask List'].apply(convert_to_numeric)
data_raw['Ask Size List'] = data_raw['Ask Size List'].apply(convert_to_numeric)


In [4]:
data_raw['Mid Price'] = data_raw.apply(lambda row: (row['Bid List'][0] + row['Ask List'][0]) / 2, axis=1)
# Calculate the volatility of the past 200 lines' volatility
data_raw['Sigma'] = data_raw['Mid Price'].rolling(200).std()
data_raw["Order_coming_rate"] =200 / data_raw['Timestamp'].diff(periods=200).dt.total_seconds()

# Drop any NaN values
data_raw.dropna(inplace=True)

In [5]:
data_raw.head()

,Timestamp,Bid List,Bid Size List,Ask List,Ask Size List,Mid Price,Sigma,Order_coming_rate
200,2024-03-06 21:54:51,"[3744.93, 3744.92, 3744.91, 3744.9, 3744.89]","[0.002, 0.002, 0.002, 0.002, 0.12794]","[3744.94, 3745.0, 3745.03, 3745.05, 3745.06]","[8.870029, 0.547, 0.001, 1.272315, 1.201]",3744.935,0.911406,9.090909
201,2024-03-06 21:54:51,"[3744.93, 3744.92, 3744.91, 3744.9, 3744.89]","[0.002, 0.002, 0.002, 0.002, 0.12894]","[3744.94, 3744.99, 3745.0, 3745.02, 3745.05]","[9.870029, 0.002, 0.545, 0.001, 0.793317]",3744.935,0.908554,9.090909
202,2024-03-06 21:54:51,"[3744.93, 3744.92, 3744.91, 3744.9, 3744.89]","[0.002, 0.002, 0.002, 0.002, 0.12894]","[3744.94, 3744.99, 3745.0, 3745.02, 3745.05]","[10.863029, 0.002, 0.545, 0.001, 0.793317]",3744.935,0.905595,9.090909
203,2024-03-06 21:54:52,"[3744.93, 3744.92, 3744.91, 3744.9, 3744.89]","[0.002, 0.002, 0.002, 0.002, 0.12894]","[3744.94, 3744.99, 3745.0, 3745.02, 3745.03]","[11.063029, 0.102, 0.545, 0.001, 0.1]",3744.935,0.902529,8.695652
204,2024-03-06 21:54:52,"[3744.93, 3744.92, 3744.91, 3744.9, 3744.89]","[0.002, 0.002, 0.002, 0.002, 0.12894]","[3744.94, 3744.99, 3745.0, 3745.02, 3745.05]","[11.063029, 0.102, 0.545, 0.001, 0.793317]",3744.935,0.899353,8.695652


In [ ]:
start_time = data_raw['Timestamp'].iloc[0]
end_time = data_raw['Timestamp'].iloc[-1]
init_bid_price_list = data_raw['Bid List'].iloc[0]
init_ask_price_list = data_raw['Ask List'].iloc[0]
init_bid_size_list = data_raw['Bid Size List'].iloc[0]
init_ask_size_list = data_raw['Ask Size List'].iloc[0]
init_sigma = data_raw['Sigma'].iloc[0]
init_gamma = 0.0001
init_kappa = 1/data_raw['Order_coming_rate'].iloc[0]
init_q =1

initial_as = ASmodel(start_time=start_time,current_time=start_time, terminal_time=end_time,
                     bid_price_list=init_bid_price_list,ask_price_list=init_ask_price_list,
                     bid_size_list=init_bid_size_list,ask_size_list=init_ask_size_list,
                     kappa=init_kappa,gamma=init_gamma,sigma=init_sigma,q=init_q)


In [ ]:
for i,row in data_raw[:5].iterrows():

    initial_as.update_model_parameters(new_current_time=row['Timestamp'],new_bid_price_list=row['Bid List'],new_ask_price_list=row['Ask List'],new_bid_size_list=row['Bid Size List'],new_ask_size_list=row['Ask Size List'],new_sigma=row['Sigma'],new_kappa=1/row['Order_coming_rate'])
    print("Mid Price: ",initial_as.calculate_mid_price())
    print("Reference Price: ",initial_as.calculate_reference_price())
    a,b,c,d = initial_as.deliver_order()
    print(a,b,c,d)

Mid Price:  3473.5550000000003
Reference Price:  3473.554788217946
3466.660507631241 3480.4490688046512 3.386471867330552 0.0
Mid Price:  3473.535
Reference Price:  3473.534789681739
3466.870239091004 3480.199340272474 3.2736350921699002 0.0
Mid Price:  3473.535
Reference Price:  3473.534791143073
3466.8702412830053 3480.199341003141 3.273634733379289 0.0
Mid Price:  3473.535
Reference Price:  3473.5347926072714
3466.8702434793026 3480.19934173524 3.2736343738855402 0.0
Mid Price:  3473.335
Reference Price:  3473.3347937580215
3466.670245205428 3479.9993423106152 3.3531517040868524 0.0


In [ ]:
initial_as.calculate_delta_t()

0.9999497260067367

In [ ]:
class Order:
    def __init__(self, limit_bid_list, limit_ask_list, limit_bid_size, limit_ask_size):
        self.limit_bid_list = limit_bid_list
        self.limit_ask_list = limit_ask_list
        self.limit_bid_size = limit_bid_size
        self.limit_ask_size = limit_ask_size

    def update_order(self, bid, ask, bid_size, ask_size):
        # Always append new bid and ask prices and sizes
        self.limit_bid_list.append(bid)
        self.limit_bid_size.append(bid_size)
        self.limit_ask_list.append(ask)
        self.limit_ask_size.append(ask_size)
        
        # Combine lists with sizes as tuples for sorting
        combined_bid = list(zip(self.limit_bid_list, self.limit_bid_size))
        combined_ask = list(zip(self.limit_ask_list, self.limit_ask_size))

        # Sort the combined lists
        combined_bid.sort(key=lambda x: x[0], reverse=True)  # Highest bids first
        combined_ask.sort(key=lambda x: x[0])  # Lowest asks first

        # If we have more than 5 bids, pop the lowest
        while len(combined_bid) > 5:
            combined_bid.pop()

        # If we have more than 5 asks, pop the highest
        while len(combined_ask) > 5:
            combined_ask.pop(-1)

        # Unpack the combined lists back into separate lists
        self.limit_bid_list, self.limit_bid_size = zip(*combined_bid)
        self.limit_ask_list, self.limit_ask_size = zip(*combined_ask)

# Example usage
limit_bid_list = [101, 102, 103, 104, 105]
limit_ask_list = [106, 107, 108, 109, 110]
limit_bid_size = [10, 9, 8, 7, 6]
limit_ask_size = [5, 4, 3, 2, 1]

# Create Order object
order = Order(limit_bid_list, limit_ask_list, limit_bid_size, limit_ask_size)

# Update order with new bid and ask
order.update_order(100, 105, 10, 10)

# Check results
print("Bid Prices:", order.limit_bid_list)
print("Ask Prices:", order.limit_ask_list)
print("Bid Sizes:", order.limit_bid_size)
print("Ask Sizes:", order.limit_ask_size)


Bid Prices: (105, 104, 103, 102, 101)
Ask Prices: (105, 106, 107, 108, 109)
Bid Sizes: (6, 7, 8, 9, 10)
Ask Sizes: (10, 5, 4, 3, 2)


In [6]:
for i in range(5):
    print(i)

0
1
2
3
4
